In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis' )

import matplotlib
import copy
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import pandas as pd
import pickle
from scipy.signal import decimate
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean, plot_one_side, calculate_error_bars
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.individual_trial_analysis_utils import ZScoredTraces, SessionData, CueAlignedData, CueAlignedSidedData, ChoiceAlignedData

In [ ]:
def get_all_experimental_records():
    experiment_record = pd.read_csv('T:\\photometry_2AC\\experimental_record.csv', dtype='str')
    experiment_record['date'] = experiment_record['date'].astype(str)
    return experiment_record

In [ ]:
def open_experiment(experiment_to_add):
    for index, experiment in experiment_to_add.iterrows():
        #saving_folder = 'S:\\users\\Yvonne\\photometry_2AC\\processed_data\\' + experiment['mouse_id'] + '\\'
        saving_folder = 'T:\\photometry_2AC\\processed_data\\' + experiment['mouse_id'] + '\\'
        #saving_folder = 'C:\\Users\\francescag\\Documents\\PhD_Project\\SNL_photo_photometry\\processed_data' + experiment['mouse_id']  + '\\'
        restructured_data_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)
        session_traces = SessionData(experiment['fiber_side'], experiment['recording_site'], experiment['mouse_id'], experiment['date'])
    return session_traces

In [ ]:
class CustomAlignedData(object):
    def __init__(self, session_data, params):
        #saving_folder = 'S:\\users\\Yvonne\\photometry_2AC\\processed_data\\' + session_data.mouse + '\\'
        saving_folder = 'T:\\photometry_2AC\\processed_data\\' + session_data.mouse + '\\'
        #saving_folder = 'C:\\Users\\francescag\\Documents\\PhD_Project\\SNL_photo_photometry\\processed_data' + session_data.mouse + '\\'
        restructured_data_filename = session_data.mouse + '_' + session_data.date + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = session_data.mouse + '_' + session_data.date + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)

        fiber_options = np.array(['left', 'right'])
        fiber_side_numeric = (np.where(fiber_options == session_data.fiber_side)[0] + 1)[0]
        contra_fiber_side_numeric = (np.where(fiber_options != session_data.fiber_side)[0] + 1)[0]

        self.ipsi_data = ZScoredTraces(trial_data, dff, params, fiber_side_numeric, 0)
        self.ipsi_data.get_peaks()
        self.contra_data = ZScoredTraces(trial_data, dff,params, contra_fiber_side_numeric, 0)
        self.contra_data.get_peaks()

In [ ]:
mouse_id = 'TS22'
date = '20230510'

In [ ]:
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data = open_experiment(experiment_to_process)

saving_folder = 'T:\\photometry_2AC\\processed_data\\' + mouse_id + '\\'
#saving_folder = 'S:\\users\\Yvonne\\photometry_2AC\\processed_data\\' + mouse_id + '\\'
#saving_folder = 'C:\\Users\\francescag\\Documents\\PhD_Project\\SNL_photo_photometry\\processed_data' + session_data.mouse + '\\'
restructured_data_filename = mouse_id + '_' + date + '_' + 'restructured_data.pkl'
trial_data = pd.read_pickle(saving_folder + restructured_data_filename)

cot_trials = trial_data[trial_data['Sound type'] == 0]

silence_trials = trial_data[trial_data['Sound type'] == 1]

num_silence_trials = silence_trials.shape[0]
per_missed_trials = silence_trials[silence_trials['Trial outcome']==3].shape[0]/num_silence_trials
per_correct_trials = silence_trials[silence_trials['Trial outcome']==1].shape[0]/num_silence_trials
per_incorrect_trials = silence_trials[silence_trials['Trial outcome']==0].shape[0]/num_silence_trials
silence_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['silence']})

num_cot_trials_trials = cot_trials.shape[0]
per_missed_trials = cot_trials[cot_trials['Trial outcome']==3].shape[0]/num_cot_trials_trials
per_correct_trials = cot_trials[cot_trials['Trial outcome']==1].shape[0]/num_cot_trials_trials
per_incorrect_trials = cot_trials[cot_trials['Trial outcome']==0].shape[0]/num_cot_trials_trials
cot_data_behavioural = pd.DataFrame({'mouse': [mouse_id], 'missed': [per_missed_trials], 'correct':[per_correct_trials], 'incorrect': [per_incorrect_trials], 'stimulus': ['tone']})

behavioural_data = pd.concat([silence_data_behavioural, cot_data_behavioural]).reset_index(drop=True)

params = {'state_type_of_interest': 5,
    'outcome': 2,
    'last_outcome': 0,  # NOT USED CURRENTLY
    'no_repeats' : 1,
    'last_response': 0,
    'align_to' : 'Time start',
    'instance': 0,
    'plot_range': [-6, 6],
    'first_choice_correct': 0,
    'cue': None}
test = CustomAlignedData(session_data, params)

_, cot_inds, _ = np.intersect1d(test.contra_data.trial_nums, cot_trials['Trial num'], return_indices=True)
_, silence_inds, _ = np.intersect1d(test.contra_data.trial_nums, silence_trials['Trial num'], return_indices=True)

peaks = test.contra_data.trial_peaks
cot_peaks = [peaks[trial] for trial in cot_inds if peaks[trial].size > 0]
silence_peaks = [peaks[trial] for trial in silence_inds  if peaks[trial].size > 0]

cot_traces = test.contra_data.sorted_traces[cot_inds, :]
silence_traces = test.contra_data.sorted_traces[silence_inds, :]

mean_cot_trace = np.mean(test.contra_data.sorted_traces[cot_inds, :], axis=0)
cot_error_bar_lower, cot_error_bar_upper = calculate_error_bars(mean_cot_trace,
                                                        cot_traces,
                                                        error_bar_method='sem')
                 
mean_silence_trace = np.mean(test.contra_data.sorted_traces[silence_inds, :], axis=0)
silence_error_bar_lower, silence_error_bar_upper = calculate_error_bars(mean_silence_trace,
                                                        silence_traces,
                                                        error_bar_method='sem')
mouse_silence_data = {'mouse': mouse_id, 'tone': [mean_cot_trace], 'tone error bar lower': [cot_error_bar_lower], 'tone error  bar upper': [cot_error_bar_upper], 
                     'silence': [mean_silence_trace], 'silence error bar lower': [silence_error_bar_lower], 'silence error bar upper': [silence_error_bar_upper], 
                     'mean tone peak': np.mean(cot_peaks), 'mean silence peak': np.mean(silence_peaks)}

all_mouse_silence_df = pd.DataFrame(mouse_silence_data)


In [ ]:
example_mouse_data = all_mouse_silence_df